<a href="https://colab.research.google.com/github/zrghassabi/LLM/blob/main/Chapter3_Demo2_Translation_FlanT5_with_Evaluation%5B1%5D.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Translation from English to Spanish using Flan-T5 and Helsinki-NLP/opus-100 Dataset

## Introduction
In this notebook, we will use the Flan-T5 model to perform translation from English to Spanish. We will use the "Helsinki-NLP/opus-100" dataset from Hugging Face, specifically the en-es subset, to train and evaluate our translation model.

In [ ]:
!pip install transformers tensorflow datasets rouge-score nltk

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 547.8/547.8 kB 3.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.8/40.8 MB 20.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 15.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.9/64.9 kB 9.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 35.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 22.4 MB/s eta 0:00:00
  Created wheel for rouge-score: filename=rouge_score-0.1.2-py3-none-any.whl size=24933 sha256=b52d7b55b3405f1c6db2f698e968cefd2004048c8bc5485a2cee2a3d5eb501c3
  Stored in directory: /root/.cache/pip/wheels/5f/dd/89/461065a73be61a532ff8599a28e9beef17985c9e9c31e541b4
Successfully built rouge-score
  Attempting uninstall: requests
    Found existing installation: requests 2.31.0
    Uninstalling requests-2.31.0:
      Successfully uninstalled requests-2.31.0
  Atte

In [ ]:
from transformers import TFAutoModelForSeq2SeqLM, AutoTokenizer
import tensorflow as tf

# Load the tokenizer and model
tokenizer = AutoTokenizer.from_pretrained("google/flan-t5-base")
model = TFAutoModelForSeq2SeqLM.from_pretrained("google/flan-t5-base")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/2.54k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.42M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/2.20k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.40k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/990M [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/tf_keras/src/initializers/initializers.py:121: UserWarning: The initializer RandomNormal is unseeded and being called multiple times, which will return identical values each time (even if the initializer is unseeded). Please update your code to provide a seed to the initializer, or avoid using the same initializer instance more than once.
  warnings.warn(
All PyTorch model weights were used when initializing TFT5ForConditionalGeneration.

All the weights of TFT5ForConditionalGeneration were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFT5ForConditionalGeneration for predictions without further training.


## Loading the Dataset

In [ ]:
from datasets import load_dataset

# Load the Helsinki-NLP/opus-100 dataset
dataset = load_dataset('Helsinki-NLP/opus-100', 'en-es')
print(dataset['train'][0])

Generating test split:   0%|          | 0/2000 [00:00<?, ? examples/s]

Generating train split:   0%|          | 0/1000000 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/2000 [00:00<?, ? examples/s]

{'translation': {'en': "It was the asbestos in here, that's what did it!", 'es': 'Fueron los asbestos aquí. ¡Eso es lo que ocurrió!'}}


## Data Preprocessing

In [ ]:
# Preprocess the dataset for input into the model
def preprocess_data(examples):
    inputs = [f'Translate from English to Spanish: {example["en"]}' for example in examples['translation']]
    targets = [example['es'] for example in examples['translation']]

    # Tokenize inputs
    model_inputs = tokenizer(inputs, max_length=128, truncation=True, padding="max_length", return_tensors="tf")

    # Tokenize targets
    labels = tokenizer(targets, max_length=128, truncation=True, padding="max_length", return_tensors="tf")

    model_inputs["labels"] = labels["input_ids"]

    # For decoder inputs
    decoder_inputs = tokenizer(targets, max_length=128, truncation=True, padding="max_length", return_tensors="tf")
    model_inputs["decoder_input_ids"] = decoder_inputs["input_ids"]

    return model_inputs


train_dataset = dataset['train'].select(range(30000)).map(preprocess_data, batched=True)
test_dataset = dataset['test'].select(range(2000)).map(preprocess_data, batched=True)

train_dataset = train_dataset.to_tf_dataset(
    columns=['input_ids', 'attention_mask', 'decoder_input_ids'],
    label_cols=['labels'],
    shuffle=True,
    batch_size=64,
    collate_fn=None
)

test_dataset = test_dataset.to_tf_dataset(
    columns=['input_ids', 'attention_mask', 'decoder_input_ids'],
    label_cols=['labels'],
    shuffle=True,
    batch_size=64,
    collate_fn=None
)

Map:   0%|          | 0/30000 [00:00<?, ? examples/s]

Map:   0%|          | 0/2000 [00:00<?, ? examples/s]

/usr/local/lib/python3.10/dist-packages/datasets/arrow_dataset.py:410: FutureWarning: The output of `to_tf_dataset` will change when a passing single element list for `labels` or `columns` in the next datasets version. To return a tuple structure rather than dict, pass a single string.
Old behaviour: columns=['a'], labels=['labels'] -> (tf.Tensor, tf.Tensor)  
             : columns='a', labels='labels' -> (tf.Tensor, tf.Tensor)  
New behaviour: columns=['a'],labels=['labels'] -> ({'a': tf.Tensor}, {'labels': tf.Tensor})  
             : columns='a', labels='labels' -> (tf.Tensor, tf.Tensor) 
  warnings.warn(


In [ ]:
model.get_layer("shared").trainable = False
model.get_layer("encoder").trainable = False
model.get_layer("decoder").trainable = False

## Model Training

In [ ]:
import tf_keras
# Compile the model
model.compile(optimizer=tf_keras.optimizers.Adam(learning_rate=2e-5),
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True))

# Train the model
model.fit(train_dataset, epochs=3)

Epoch 1/3


Cause: for/else statement not yet supported
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


Cause: for/else statement not yet supported
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
469/469 [==============================] - 264s 362ms/step - loss: 48.3423
Epoch 2/3
469/469 [==============================] - 170s 362ms/step - loss: 47.3018
Epoch 3/3
469/469 [==============================] - 170s 362ms/step - loss: 46.3005


## Calculating ROUGE and BLEU Scores

1. Imports and Setup:

ROUGE: The code imports rouge_scorer from the rouge_score library to calculate ROUGE scores, which measure overlap between reference and generated text.

BLEU: It uses sentence_bleu from the nltk.translate.bleu_score library to calculate BLEU scores, a metric used to evaluate the quality of machine-translated text by comparing n-grams between the hypothesis (generated translation) and the reference (actual translation).
NLTK: The code downloads the punkt tokenizer using nltk.download('punkt'), which is necessary for tokenizing the text in the BLEU score calculation.

2. Translation Function:

This function takes tokenized input (generated by the model) and:

Generates a translation using the model.generate() method. The arguments used are:
max_length=128: limits the generated translation to 128 tokens.
num_beams=4: uses beam search with 4 beams to improve translation quality by exploring multiple output paths.
early_stopping=True: stops the generation early when it predicts the end of the sentence.
Decodes the generated tokens into a human-readable string using tokenizer.decode(), removing any special tokens like padding.

3. Scoring Function:

This function calculates the ROUGE and BLEU scores for comparing a reference translation with the generated hypothesis:

ROUGE: It initializes the RougeScorer to calculate ROUGE-1, ROUGE-2, and ROUGE-L scores:
ROUGE-1: Measures overlap of unigrams (single words).
ROUGE-2: Measures overlap of bigrams (pairs of words).
ROUGE-L: Measures the longest common subsequence (LCS) between the reference and hypothesis.

BLEU: Uses sentence_bleu with a smoothing function (method4) to avoid zero BLEU scores, especially for short sequences.

The text is tokenized using NLTK's word_tokenize() to break down the sentences into words.
The BLEU score compares the hypothesis' tokens to the reference tokens, which helps determine how well the translation matches the reference.

4. Evaluation of Translation:

Batch Processing: The next(iter(test_dataset)) retrieves a batch from the test_dataset, which will be used as input for translation.
Translation: The translate() function is called on this batch to generate a translated text.
Reference Text: The original text (reference) is extracted from the dataset using tokenizer.decode().


5. Scoring the Translation:

The calculate_scores() function is used to compute the ROUGE and BLEU scores by comparing the reference and translated text.


6. Printing the Results:

The code prints the reference translation, the generated translation, and the calculated ROUGE and BLEU scores.
Example Flow:
Input: A batch from the test_dataset.
Process: The model generates a translation, which is compared to the reference translation from the dataset.
Output: The function prints the ROUGE-1, ROUGE-2, ROUGE-L scores, and the BLEU score, showing how well the generated translation matches the reference.
This is a standard approach to evaluate machine translation models, providing both n-gram overlap (BLEU) and sequence overlap (ROUGE) metrics.






In [ ]:
from rouge_score import rouge_scorer
from nltk.translate.bleu_score import sentence_bleu, SmoothingFunction
import nltk
nltk.download('punkt')

def translate(inputs):
    outputs = model.generate(inputs[0]["input_ids"], max_length=128, num_beams=4, early_stopping=True)
    return tokenizer.decode(outputs[0], skip_special_tokens=True)


# Function to calculate ROUGE and BLEU scores
def calculate_scores(reference, hypothesis):
    # Initialize scorers
    rouge = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeL'], use_stemmer=True)
    bleu_smoothing = SmoothingFunction().method4

    # Calculate ROUGE scores
    rouge_scores = rouge.score(reference, hypothesis)

    # Calculate BLEU score
    reference_tokens = [nltk.word_tokenize(reference)]
    hypothesis_tokens = nltk.word_tokenize(hypothesis)
    bleu_score = sentence_bleu(reference_tokens, hypothesis_tokens, smoothing_function=bleu_smoothing)

    return rouge_scores, bleu_score

# Evaluate translations and calculate scores
batch = next(iter(test_dataset))
translated_text = translate(batch)
reference_text = tokenizer.decode(batch[1][0], skip_special_tokens=True)
rouge_scores, bleu_score = calculate_scores(reference_text, translated_text)
print(f"Reference: {reference_text}")
print(f"Translation: {translated_text}")
print(f"ROUGE Scores: {rouge_scores}")
print(f"BLEU Score: {bleu_score}")
print()

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


Reference: - Estaba limpiando.
Translation: Estaba limpiando.
ROUGE Scores: {'rouge1': Score(precision=1.0, recall=1.0, fmeasure=1.0), 'rouge2': Score(precision=1.0, recall=1.0, fmeasure=1.0), 'rougeL': Score(precision=1.0, recall=1.0, fmeasure=1.0)}
BLEU Score: 0.41252121275142684



## Conclusion
In this notebook, we used the Flan-T5 model to perform translation from English to Spanish using the Helsinki-NLP/opus-100 dataset. We preprocessed the dataset, fine-tuned the model, and performed translations. We calculated ROUGE and BLEU scores to assess the quality of the translations. The results demonstrate the effectiveness of the Flan-T5 model for translation tasks.